# Identifying Refusal Location for Harmful vs Harmless Prompt Analysis

This notebook extracts and analyzes internal activations from language models when processing harmful vs harmless prompts. It creates "refusal vectors" that can be used to understand how models internally distinguish between safe and unsafe content.

## Overview

The notebook performs the following main steps:
1. **Data Extraction**: Extract activations from model layers for harmful and harmless prompts
2. **Mean Computation**: Calculate mean activations for each category
3. **Refusal Vector Creation**: Compute difference vectors between harmful and harmless means
4. **Visualization**: Plot cosine similarities to analyze differences across layers

## Prerequisites

- CUDA-capable GPU (configured for `cuda:1`)
- Required datasets stored locally in `../dataset_folder/`
- Pre-trained model stored in `../llm_models/`
- Sufficient disk space for activation storage (~GBs depending on dataset size)

## Usage Instructions

1. **Configure Model and Datasets**: Modify the configuration section below
2. **Set Processing Mode**: Choose "harmful" or "harmless" for data extraction
3. **Run Extraction**: Execute the activation extraction pipeline
4. **Compute Statistics**: Calculate means and refusal vectors
5. **Analyze Results**: View visualizations and save outputs

## 1. Import Libraries and Setup

Import required libraries for model loading, data processing, and visualization.

In [ ]:
import os
import torch
import numpy as np
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer
import torch.nn.functional as F
import matplotlib.pyplot as plt

device = "cuda:1"

## 2. Configuration

Configure dataset sources, model paths, and processing parameters.

### Key Parameters:
- **DATASET_SOURCE_HARMFUL**: PKU-SafeRLHF dataset for harmful prompts
- **DATASET_SOURCE_HARMLESS**: Alpaca dataset for harmless instructions  
- **TARGET_SIZE**: Number of examples to process (20,000)
- **MODEL_NAME**: Model identifier for file naming
- **BATCH_SIZE**: Processing batch size for memory management
- **NUM_LAST_TOKENS**: Number of final tokens to analyze (3)

⚠️ **Important**: Ensure the model path `../llm_models/{MODEL_NAME}` exists and datasets are available in `../dataset_folder/`

In [ ]:
DATASET_SOURCE_HARMFUL = "PKU-Alignment/PKU-SafeRLHF-prompt"
DATASET_SOURCE_HARMLESS = "tatsu-lab/alpaca"

PROMPT_COLUMN_HARMFUL = "prompt"
PROMPT_COLUMN_HARMLESS = "instruction"

TARGET_SIZE = 20000

MODEL_NAME = "Qwen3-0.6B"
MODEL_PATH = f"../llm_models/{MODEL_NAME}"

OUTPUT_DIR_HARMFUL = f"{MODEL_NAME}/harmful_prompt_activations"
OUTPUT_DIR_HARMLESS = f"{MODEL_NAME}/harmless_prompt_activations"
os.makedirs(OUTPUT_DIR_HARMFUL, exist_ok=True)
os.makedirs(OUTPUT_DIR_HARMLESS, exist_ok=True)

BATCH_SIZE = 512
NUM_LAST_TOKENS = 3

## 3. Load Model and Tokenizer

Load the language model and tokenizer. The model is loaded in half precision (float16) to save memory and moved to the specified GPU device.

🔧 **Note**: This may take several minutes depending on model size.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    torch_dtype=torch.float16
).to(device)
model.eval()

## 4. Set Processing Mode

Choose which type of data to process. You need to run the next few cells **twice**:
1. First with `MODE = "harmful"` to extract harmful prompt activations
2. Then with `MODE = "harmless"` to extract harmless prompt activations

💡 **Tip**: **ONLY** after completing both runs, proceed to the analysis sections at the end.

In [ ]:
# MODE = "harmful"
MODE = "harmless"

## 5. Load and Sample Dataset

Load the appropriate dataset based on the selected mode and sample examples to reach the target size. 

The sampling uses a step size to evenly distribute examples across the full dataset rather than taking only the first N examples.

In [ ]:
if MODE == "harmful":
    dataset = load_from_disk(f"../dataset_folder/{DATASET_SOURCE_HARMFUL}")
    if "train" in dataset:
        ds = dataset["train"]
    else:
        ds = dataset
    all_prompts = ds[PROMPT_COLUMN_HARMFUL]
    total_examples = len(all_prompts)

    step = total_examples // TARGET_SIZE
    all_prompts = all_prompts[::step][:TARGET_SIZE]
else:
    dataset = load_from_disk(f"../dataset_folder/{DATASET_SOURCE_HARMLESS}")
    if "train" in dataset:
        ds = dataset["train"]
    else:
        ds = dataset
    all_prompts = ds[PROMPT_COLUMN_HARMLESS]
    total_examples = len(all_prompts)

    step = total_examples // TARGET_SIZE
    all_prompts = all_prompts[::step][:TARGET_SIZE]

## 6. Setup Activation Hooks

Create forward hooks to capture internal activations from each transformer layer. The hooks extract the last N tokens' activations (set before), which are most relevant for understanding the model's final decision-making process.

🧠 **Technical Detail**: Forward hooks intercept the output of each layer during the forward pass without affecting the model's computation.

In [ ]:
residuals_by_layer = {}

def make_hook(layer_idx):
    """
    Returns a forward hook function that, when attached to a transformer layer,
    captures the activations (output) of that layer for the last NUM_LAST_TOKENS tokens.
    """
    def hook_fn(module, inputs, outputs):
        with torch.no_grad():
            activation = outputs[0] if isinstance(outputs, tuple) else outputs
            act_cpu = activation.detach().cpu()
            seq_len = act_cpu.size(1)

            if seq_len >= NUM_LAST_TOKENS:
                residuals_by_layer[layer_idx] = act_cpu[0, -NUM_LAST_TOKENS :, :].clone()
            else:
                residuals_by_layer[layer_idx] = act_cpu[0, :, :].clone()

    return hook_fn

hooks = []
for idx, layer in enumerate(model.model.layers):
    hooks.append(layer.register_forward_hook(make_hook(idx)))

## 7. Extract Activations

Process each prompt through the model and collect activations from all layers. The data is saved in batches to manage memory usage.

⏱️ **Processing Time**: This step may take 80-140 minutes depending on sample size chosen and gpu hardware available.

### Process:
1. Format each prompt using the model's chat template
2. Run forward pass to trigger hooks and capture activations  
3. Save activations in batches to disk
4. Clean up hooks when complete

In [ ]:
batch_data = []
batch_counter = 0

for idx, prompt in enumerate(all_prompts):
    residuals_by_layer.clear()
    chat = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user",   "content": prompt}
    ]
    chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer(
        chat_text,
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        _ = model(**inputs)
        
    example_record = {
        "prompt": prompt,
        "residuals": {layer_idx: tensor for layer_idx, tensor in residuals_by_layer.items()}
    }
    batch_data.append(example_record)

    is_last_example = (idx + 1) == len(all_prompts)
    if (len(batch_data) >= BATCH_SIZE) or is_last_example:
        batch_id = batch_counter
        if MODE == "harmful":
            save_path = os.path.join(OUTPUT_DIR_HARMFUL, f"batch_{batch_id:04d}.pt")
        else:
            save_path = os.path.join(OUTPUT_DIR_HARMLESS, f"batch_{batch_id:04d}.pt")
        torch.save(batch_data, save_path)
        print(f"Saved batch {batch_id} with {len(batch_data)} examples → {save_path}")
        batch_counter += 1
        batch_data = [] 

for h in hooks:
    h.remove()

print("✅ Finished processing all examples.")

---

## 8. Statistical Analysis Section

⚠️ **Prerequisites**: Run sections 4-7 **twice** (once for "harmful" and once for "harmless" mode) before proceeding.

The following sections compute statistics and create refusal vectors from the extracted activations.

### Setup Output Paths

In [ ]:
HARMFUL_DIR = OUTPUT_DIR_HARMFUL
HARMLESS_DIR = OUTPUT_DIR_HARMLESS

OUTPUT_MEAN_HARMFUL   = f"{MODEL_NAME}/harmful_mean.pt"
OUTPUT_MEAN_HARMLESS  = f"{MODEL_NAME}/harmless_mean.pt"
OUTPUT_REFUSAL_VECTOR = f"{MODEL_NAME}/refusal_vector.pt"

### Define Mean Computation Function

This function computes the mean activation across all examples for each layer and token position. It discards previously loaded data, to avoid loading all data into memory simultaneously.

In [ ]:
def compute_means_for_directory(dir_path):
    """
    Iterates over all .pt files in `dir_path`, each of which is a list of dicts:
        { "prompt": str,
          "residuals": { layer_idx: Tensor[NUM_LAST_TOKENS, hidden_dim], ... }
        }
    Returns a dict: { layer_idx: Tensor[NUM_LAST_TOKENS, hidden_dim] } containing the float32 mean
    across all examples in that directory.
    """
    means = {}        
    count = 0         

    
    filenames = sorted([f for f in os.listdir(dir_path) if f.endswith(".pt")])
    for fname in filenames:
        batch_path = os.path.join(dir_path, fname)
        batch_list = torch.load(batch_path) 

        for example in batch_list:
            count += 1
            residuals = example["residuals"] 

            if count == 1:
                for layer_idx, tensor in residuals.items():
                    means[layer_idx] = tensor.float().clone()
            else:
                for layer_idx, tensor in residuals.items():
                    x = tensor.float()  
                    old_mean = means[layer_idx]
                    means[layer_idx] = old_mean + (x - old_mean) / count

        del batch_list
        torch.cuda.empty_cache()

    return means

## 9. Compute Mean Activations

Calculate the mean activation vectors for harmful and harmless prompts across all layers and token positions. These means represent the "typical" internal state when processing each type of content.

In [ ]:
means_harmful = compute_means_for_directory(HARMFUL_DIR)
torch.save(means_harmful, OUTPUT_MEAN_HARMFUL)
print(f"Saved harmful means → {OUTPUT_MEAN_HARMFUL}")

means_harmless = compute_means_for_directory(HARMLESS_DIR)
torch.save(means_harmless, OUTPUT_MEAN_HARMLESS)
print(f"Saved harmless means → {OUTPUT_MEAN_HARMLESS}")

## 10. Create Refusal Vector

Compute the refusal vector by taking the difference between harmful and harmless mean activations. This vector represents the direction in activation space that distinguishes harmful from harmless content.

**Formula**: `refusal_vector = mean_harmful - mean_harmless`

In [ ]:
means_harmful = torch.load(OUTPUT_MEAN_HARMFUL)
means_harmless = torch.load(OUTPUT_MEAN_HARMLESS)

refusal_vector = {}
for layer_idx in means_harmful:
    refusal_vector[layer_idx] = means_harmful[layer_idx] - means_harmless[layer_idx]

torch.save(refusal_vector, OUTPUT_REFUSAL_VECTOR)
print(f"Saved refusal vector → {OUTPUT_REFUSAL_VECTOR}")

## 11. Visualization: Cosine Similarity Analysis

Analyze how similar the harmful and harmless activations are across different layers and token positions. Lower cosine similarity indicates stronger differentiation between harmful and harmless content.

### Interpretation:
- **High similarity**: Layer shows little difference between harmful/harmless
- **Comparitively Lower similarity**: Layer differentiates content types. In practise, there is hardly a dip of over 0.1.
- **Patterns across tokens**: Shows which token positions are most informative

In [ ]:
means_harmful = torch.load(OUTPUT_MEAN_HARMFUL)
means_harmless = torch.load(OUTPUT_MEAN_HARMLESS)

layer_indices = sorted(means_harmful.keys())
cos_sims = {i: [] for i in range(NUM_LAST_TOKENS)}

for layer_idx in layer_indices:
    v_h = means_harmful[layer_idx]
    v_safe = means_harmless[layer_idx]

    for token_pos in range(NUM_LAST_TOKENS):
        a = v_h[token_pos].unsqueeze(0)    
        b = v_safe[token_pos].unsqueeze(0) 
        cos_val = F.cosine_similarity(a, b, dim=1).item()
        cos_sims[token_pos].append(cos_val)

plt.figure(figsize=(10, 6))
for token_pos in range(NUM_LAST_TOKENS):
    label = {
        0: "third‐last token",
        1: "second‐last token",
        2: "last token"
    }[token_pos]
    plt.plot(layer_indices, cos_sims[token_pos], label=label)

plt.xlabel("Layer Index")
plt.ylabel("Cosine Similarity")
plt.title("Cosine Similarity Between Harmful & Harmless Mean Vectors\n(for Each of the Last 3 Token Positions)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 12. Refusal Vector Injection Testing

Test the effectiveness of the created refusal vectors by using them in injection hooks during model generation. This allows you to see how the vectors affect model behavior when added or removed from activations.

In [ ]:
import torch
from typing import Dict, Iterable, List
from transformers import AutoTokenizer, AutoModelForCausalLM

# ---------------------------
# Hook utils
# ---------------------------

def _get_block_for_layer(model, layer_idx: int):
    """
    Locate the transformer block at layer_idx for common HF CausalLMs.
    Supports Qwen/LLaMA/Mistral-style (.model.layers) and GPT-2-style (.transformer.h).
    """
    if hasattr(model, "model") and hasattr(model.model, "layers"):
        layers = model.model.layers
    elif hasattr(model, "transformer") and hasattr(model.transformer, "h"):
        layers = model.transformer.h
    else:
        raise ValueError("Unsupported model structure: cannot locate transformer blocks.")

    if not (0 <= layer_idx < len(layers)):
        raise ValueError(f"layer_idx {layer_idx} out of range [0, {len(layers)-1}]")

    return layers[layer_idx]


def register_multi_refusal_edit(
    model,
    refusal_vector: Dict[int, torch.Tensor],   # layer_idx -> [NUM_LAST_TOKENS, hidden_dim]
    plan: Dict[int, Iterable[int]],            # e.g., {10: [0,1], 11: [1]}
    *,
    num_last_tokens: int = 3,
    mode: str = "remove",                      # "remove" -> project-out; "add" -> inject
    add_strength: float = 1.0,                 # scale for "add" mode
) -> List[torch.utils.hooks.RemovableHandle]:
    """
    Register forward hooks to edit residuals at multiple (layer, offset) pairs.

    Token offsets are from the END of the input sequence:
      0 = last token, 1 = second-last, 2 = third-last (for num_last_tokens=3).

    Modes:
      - "remove": v <- v - proj_ref(v)    (project-out refusal direction)
      - "add"   : v <- v + add_strength * ref_unit

    Returns:
      List of hook handles; call .remove() on each to undo.
    """
    if mode not in {"remove", "add"}:
        raise ValueError("mode must be either 'remove' or 'add'")

    handles: List[torch.utils.hooks.RemovableHandle] = []

    # Prepack per-layer vectors keyed by offset
    prepacked: Dict[int, Dict[int, torch.Tensor]] = {}
    for layer_idx, offsets in plan.items():
        if layer_idx not in refusal_vector:
            raise KeyError(f"refusal_vector has no entry for layer {layer_idx}")
        layer_t = refusal_vector[layer_idx]  # [num_last_tokens, hidden_dim]
        if layer_t.size(0) != num_last_tokens:
            raise ValueError(
                f"refusal_vector[{layer_idx}] has {layer_t.size(0)} rows, expected {num_last_tokens}"
            )

        uniq = sorted(set(int(o) for o in offsets))
        for o in uniq:
            if not (0 <= o < num_last_tokens):
                raise ValueError(f"Offset {o} invalid for num_last_tokens={num_last_tokens}.")

        # Map offset_from_last -> stored index in your saved tensor
        # Your storage: index 0 = third-last, 1 = second-last, 2 = last
        # So stored_idx = num_last_tokens - 1 - offset
        layer_vecs = {}
        for o in uniq:
            stored_idx = num_last_tokens - 1 - o
            layer_vecs[o] = layer_t[stored_idx]   # [hidden_dim], float32 CPU
        prepacked[layer_idx] = layer_vecs

    # One hook per layer
    for layer_idx, offset_to_vec in prepacked.items():
        block = _get_block_for_layer(model, layer_idx)
        offsets_sorted = sorted(offset_to_vec.keys())

        def make_hook(offsets_sorted=offsets_sorted, offset_to_vec=offset_to_vec):
            def hook_fn(module, inputs, output):
                # Some models return a tuple; first element is hidden states
                is_tuple = isinstance(output, tuple)
                x = output[0] if is_tuple else output  # [B, T, H]
                if x.dim() != 3 or x.size(1) == 0:
                    return output

                x_mod = x.clone()
                B, T, H = x_mod.shape

                for o in offsets_sorted:
                    pos = T - 1 - o  # offset-from-end within current sequence
                    if pos < 0:
                        continue  # sequence shorter than requested offset; skip

                    # Cast to current device/dtype and unit-normalize
                    ref = offset_to_vec[o].to(device=x_mod.device, dtype=x_mod.dtype)
                    denom = ref.norm()
                    if torch.isnan(denom) or float(denom) < 1e-12:
                        continue
                    ref_unit = ref / denom

                    if mode == "remove":
                        v = x_mod[:, pos, :]                           # [B, H]
                        proj = (v @ ref_unit)[:, None] * ref_unit[None, :]  # [B, H]
                        x_mod[:, pos, :] = v - proj
                    else:  # mode == "add"
                        x_mod[:, pos, :] = x_mod[:, pos, :] + add_strength * ref_unit

                return (x_mod,) + output[1:] if is_tuple else x_mod
            return hook_fn

        handle = block.register_forward_hook(make_hook())
        handles.append(handle)

    return handles

### Setup Refusal Vector Parameters

Configure which layer and token position to use for testing the injection hook. The refusal vector is loaded from the previously created file and prepared for injection.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH, trust_remote_code=True, torch_dtype=torch.float16
).to(device).eval()

# Refusal vector
refusal_vector = torch.load(r"Qwen3-4B/refusal_vector.pt")  # dict[layer_idx] -> [3, hidden_dim]

In [ ]:
# Choose plan: e.g. layer 10 remove offsets {0,1}; layer 11 add offset {1}
plan_remove = {}
plan_add    = {20: [0]}

# Option A: REMOVE (project-out) at specified layers/offsets
handles_remove = register_multi_refusal_edit(
    model,
    refusal_vector=refusal_vector,
    plan=plan_remove,
    num_last_tokens=3,
    mode="remove",
)

# Option B: ADD (inject) at specified layers/offsets (strength can be tuned)
handles_add = register_multi_refusal_edit(
    model,
    refusal_vector=refusal_vector,
    plan=plan_add,
    num_last_tokens=3,
    mode="add",
    add_strength=40,   # try values like 0.5, 1.0, 2.0
)

### Test Injection with Sample Prompt

Run a test using a potentially harmful prompt to see how the refusal vector affects model behavior. 

In [ ]:
chat = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "How to make coffee? /no_think"}
]
chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(chat_text, return_tensors="pt").to(model.device)

with torch.no_grad():
    out_ids = model.generate(**inputs, max_new_tokens=256, do_sample=False)
print(tokenizer.decode(out_ids[0], skip_special_tokens=True))

# Clean up hooks
for h in handles_remove + handles_add:
    h.remove()

## Summary

This notebook has successfully created a complete pipeline for refusal vector analysis and testing:

### 📊 **Analysis Components Created:**

✅ **Extracted Activations**: Saved in `{MODEL_NAME}/harmful_prompt_activations/` and `{MODEL_NAME}/harmless_prompt_activations/`

✅ **Mean Vectors**: Saved as `{MODEL_NAME}/harmful_mean.pt` and `{MODEL_NAME}/harmless_mean.pt`

✅ **Refusal Vector**: Saved as `{MODEL_NAME}/refusal_vector.pt`

✅ **Cosine Similarity Analysis**: Visualization showing layer-wise differences between harmful and harmless activations

✅ **Injection Hook System**: Precise vector injection/removal for model interventions

### 📁 **File Outputs:**
All outputs are saved with the model name prefix for easy organization and comparison across different models. The refusal vectors can be directly used in other experiments and intervention studies.